# Unzip archivos

In [4]:
!unzip ./drive/MyDrive/KalokagathIA/results.zip

Archive:  ./drive/MyDrive/KalokagathIA/results.zip
  inflating: logs/1621385657.7439206/events.out.tfevents.1621385657.eeb83ccd37f3.24.1  
  inflating: logs/events.out.tfevents.1621385657.eeb83ccd37f3.24.0  
  inflating: modelo_final/config.json  
  inflating: modelo_final/pytorch_model.bin  
  inflating: modelo_final/training_args.bin  


# Carga del modelo

In [8]:
!pip install transformers

     |████████████████████████████████| 2.3MB 29.6MB/s 
     |████████████████████████████████| 3.3MB 43.3MB/s 
     |████████████████████████████████| 901kB 41.2MB/s 


In [10]:
from transformers import BertForSequenceClassification, AutoTokenizer

model_directory = './modelo_final/'

model = BertForSequenceClassification.from_pretrained(model_directory)

In [ ]:
model.eval()

# Carga del dataset

In [19]:
import pandas as pd

# OJO que hay textos duplicados
data_path = './drive/MyDrive/KalokagathIA/validacion.csv'

columns = ['Autor', 'Obra', 'Fragmento', 'Texto']

df = pd.DataFrame(columns=columns)

with open(data_path, 'r') as file:
  line = file.read().split('\n')
  for l in line:
    try:
      data = l.split(';')
      autor = data[0]
      obra = data[1]
      fragmento = data[2]
      texto = ';'.join(data[3:])
      data_dict = {'Autor': autor, 'Obra': obra, 'Fragmento': fragmento, 'Texto': texto}
      df = df.append(data_dict, ignore_index=True)
    except:
      print(l)

print(df.head())
print(df.describe())


       Autor  ...                                              Texto
0  aristotle  ...  Γ.Ὅτι δ’ οὐκ ἔστιν αἴσθησις ἑτέρα παρὰ τὰς πέν...
1  aristotle  ...  Γ.Ὅτι δ’ οὐκ ἔστιν αἴσθησις ἑτέρα παρὰ τὰς πέν...
2  aristotle  ...  Ἐπεὶ δ’ αἰσθανόμεθα ὅτι ὁρῶμεν καὶ ἀκούομεν, ἀ...
3  aristotle  ...  Ἐπεὶ δὲ δύο διαφοραῖς ὁρίζονται μάλιστα τὴν ψυ...
4  aristotle  ...  Περὶ δὲ τοῦ μορίου τοῦ τῆς ψυχῆς ᾧ γινώσκει τε...

[5 rows x 4 columns]
            Autor   Obra Fragmento  Texto
count        1531   1531      1531   1531
unique          8     10      1408   1528
top     Euripides  Menea         1  αἰαῖ.
freq         1408   1408        10      2


## Limpieza dataset

In [32]:
import unicodedata

def strip_accents_and_lowercase(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn').lower()

In [33]:
df.Texto = df.Texto.apply(lambda x: strip_accents_and_lowercase(x))

In [41]:
df

,Autor,Obra,Fragmento,Texto
0,aristotle,deanimab3,3,γ.οτι δ’ ουκ εστιν αισθησις ετερα παρα τας πεν...
1,aristotle,deanimab3,1,γ.οτι δ’ ουκ εστιν αισθησις ετερα παρα τας πεν...
2,aristotle,deanimab3,2,"επει δ’ αισθανομεθα οτι ορωμεν και ακουομεν, α..."
3,aristotle,deanimab3,3,επει δε δυο διαφοραις οριζονται μαλιστα την ψυ...
4,aristotle,deanimab3,4,περι δε του μοριου του της ψυχης ω γινωσκει τε...
...,...,...,...,...
1526,Demosthenes,Against Dionysodorus,46,ταυτʼ εχοντες τα δικαια εισεληλυθαμεν προς υμα...
1527,Demosthenes,Against Dionysodorus,47,"εν τω ροδιων δικαστηριω εκρινομεθα, ισως αν ου..."
1528,Demosthenes,Against Dionysodorus,48,"χωρις δε τουτων, ανδρες αθηναιοι, μη αγνοειτε,..."
1529,Demosthenes,Against Dionysodorus,49,"ει μεντοι εξεσται τοις ναυκληροις, συγγραφην γ..."


# Interpreter

In [ ]:
!pip install transformers-interpret

In [42]:
# https://github.com/cdpierse/transformers-interpret
from transformers import AutoTokenizer
from transformers_interpret import SequenceClassificationExplainer

tokenizer = AutoTokenizer.from_pretrained('nlpaueb/bert-base-greek-uncased-v1')
interpreter = SequenceClassificationExplainer(model, tokenizer)

In [44]:
# ;ψευδόμενοι· σὲ δʼ ἔτικτε πατὴρ ἀνδρῶν τε θεῶν τε
print(df.loc[40].Texto)
interpretacion = interpreter(df.loc[40].Texto)
interpreter.visualize()

ψευδομενοι· σε δʼ ετικτε πατηρ ανδρων τε θεων τε


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
2,LABEL_2 (0.04),LABEL_2,0.00,[CLS] ψευδο ##μενοι · σε [UNK] ετι ##κτ ##ε πατηρ ανδρων τε θεων τε [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
2,LABEL_2 (0.04),LABEL_2,0.00,[CLS] ψευδο ##μενοι · σε [UNK] ετι ##κτ ##ε πατηρ ανδρων τε θεων τε [SEP]


In [48]:
interpretacion = interpreter('δʼ')
interpreter.visualize()
result_ids = tokenizer.tokenize('ψευδομενοι· σε δʼ ετικτε πατηρ ανδρων τε θεων τε')

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
26,LABEL_26 (0.04),LABEL_26,1.00,[CLS] [UNK] [SEP]


In [49]:
result_ids

['ψευδο',
 '##μενοι',
 '·',
 'σε',
 '[UNK]',
 'ετι',
 '##κτ',
 '##ε',
 'πατηρ',
 'ανδρων',
 'τε',
 'θεων',
 'τε']

In [ ]:
encoding = tokenizer(df.loc[0].Texto, max_length=512)